### Exercise 3.1

#### 3.1 a

In [ ]:
import pulp as pl
from ASS_3_data_3_1 import J, p, r, d, c, M

# the decision variables
S   = pl.LpVariable.dicts("S",   J, lowBound=0)                  # start times
Z   = pl.LpVariable.dicts("Z",   J, lowBound=0)                  # tardiness
Yij = pl.LpVariable.dicts("Y", [(i,j) for i in J for j in J if i!=j], 0, 1, pl.LpBinary)  # order

m = pl.LpProblem("SingleMachine_Tardiness", pl.LpMinimize)

# Objective is to minimize sum of (weighted) tardiness
m += pl.lpSum(c[j]*Z[j] for j in J)

# Release times
for j in J:
    m += S[j] >= r[j]

# No overlap via order variables (Lecture 9 slide 29)
for i in J:
    for j in J:
        if i == j: 
            continue
        m += S[i] + p[i] <= S[j] + M*(1 - Yij[(i,j)])

# Exactly one order per unordered pair (Lecture 9 slide 29)
for i in J:
    for j in J:
        if i < j:
            m += Yij[(i,j)] + Yij[(j,i)] == 1

# Tardiness linearization (Lecture 9 slide 30)
for j in J:
    m += Z[j] >= S[j] + p[j] - d[j]
    m += Z[j] >= 0

# Solve
m.solve(pl.PULP_CBC_CMD(msg=False))

# Report
sched = sorted([(j, pl.value(S[j]), pl.value(S[j])+p[j], pl.value(Z[j])) for j in J], key=lambda x: x[1])
print("Job  Start  Compl  Tardiness")
for row in sched:
    print(f"{int(row[0]):>3}  {row[1]:>5.0f}  {row[2]:>5.0f}  {row[3]:>9.0f}")
print("Total tardiness:", int(sum(z for _,_,_,z in sched)))

Job  Start  Compl  Tardiness
  9      0      2          0
  6      2      3          0
  7      2      2          0
  1      3      7          0
  2      7     12          0
  3     12     15          0
  5     15     22          2
  4     22     27          2
  8     27     30          0
 10     30     40         20
Total tardiness: 24


With only processing times and release/due dates, the optimal sequence keeps most jobs close to their due dates; lateness is limited. The reason total tardiness is only 24, means the solver can freely interleave types, so it chases tight due dates (EDD-like) right after releases, minimizing spillover. 

#### 3.1 b

In [6]:
# Sieve types (from question): {1,3,4,6,10} = type 1 ; others type 2
type1 = {1,3,4,6,10}
type_of = {j: (1 if j in type1 else 2) for j in J}

# Sequence-dependent setup: 1 hour if types differ, else 0
s = {(i,j): (1 if type_of[i] != type_of[j] else 0) for i in J for j in J if i!=j}

S   = pl.LpVariable.dicts("S",   J, lowBound=0)
Z   = pl.LpVariable.dicts("Z",   J, lowBound=0)
Yij = pl.LpVariable.dicts("Y", [(i,j) for i in J for j in J if i!=j], 0, 1, pl.LpBinary)

m = pl.LpProblem("SingleMachine_Tardiness_WithChangeovers", pl.LpMinimize)
m += pl.lpSum(c[j]*Z[j] for j in J)

for j in J:
    m += S[j] >= r[j]

# No overlap now includes setup time s[i,j] if i precedes j
for i in J:
    for j in J:
        if i == j: 
            continue
        m += S[i] + p[i] + s[(i,j)] <= S[j] + M*(1 - Yij[(i,j)])

for i in J:
    for j in J:
        if i < j:
            m += Yij[(i,j)] + Yij[(j,i)] == 1

for j in J:
    m += Z[j] >= S[j] + p[j] - d[j]
    m += Z[j] >= 0

m.solve(pl.PULP_CBC_CMD(msg=False))

sched = sorted([(j, type_of[j], pl.value(S[j]), pl.value(S[j])+p[j], pl.value(Z[j])) for j in J],
               key=lambda x: x[2])
print("Job Type Start Compl Tard")
for j,t,s,cpl,td in sched:
    print(f"{j:>3}  {t:>4} {s:>5.0f} {cpl:>5.0f} {td:>4.0f}")
print("Total tardiness:", int(sum(td for *_,td in sched)))

Job Type Start Compl Tard
  6     1     0     1    0
  9     2     2     4    0
  2     2     4     9    0
  7     2     4     4    0
  1     1    10    14    3
  3     1    14    17    0
  4     1    17    22    0
  5     2    23    30   10
  8     2    30    33    3
 10     1    34    44   24
Total tardiness: 40


When 1-hour sieve changeovers are introduced, the machine must spend additional time switching between job types, which pushes several tasks past their due dates. As a result, total tardiness increases from 24 in the no-changeover case to 40, about a 67% rise. This jump occurs because the solver now prefers to batch jobs of the same type together to avoid frequent setups, but this batching delays some time sensitive jobs of other types. In essence, the system sacrifices punctuality to reduce setup time, highlighting the trade-off between minimizing changeovers and meeting due dates.